In [1]:
%matplotlib inline
import pickle
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
np.random.seed(100)
X,y_hr,y_participant,y_activity,y_time = pickle.load(open('../data/heart_rate_tabular_data_ppg_dalia.p','rb'))
activity_dict1 = {'No Label':-1,'Sitting':0,'Stairs':1,'Soccer':2,
                'Cycling':3,'Driving':4,'Lunch':-1,'Walking':5,
                'Working':-1}
y_activity = np.array([activity_dict1[a] for a in y_activity]) 
X = X[:,:,1:].reshape(-1,256,3)

In [2]:
def get_train_test_val_data(X_hr,y_participant,y_activity,y_time):
    all_data = pd.DataFrame({'x':np.arange(X_hr.shape[0]),'y':y_participant,'activity':y_activity,'time':y_time})
    train_percentage = .7
    val_percentage = train_percentage+.1
    def split_data(df):
        df = df.sort_values('time').reset_index(drop=True)
        n = df.shape[0]
        train_index = df['x'].values[:int(n*train_percentage)]
        val_index = df['x'].values[int(n*train_percentage):int(n*val_percentage)]
        test_index = df['x'].values[int(n*val_percentage):]
        return pd.DataFrame({'train':[list(train_index)],'val':[list(val_index)],'test':[list(test_index)]})

    index_df = all_data.groupby(['y','activity'],as_index=False).apply(split_data)

    from functools import reduce
    train_index = np.array(reduce(lambda a,b:a+b,index_df['train'].values))
    val_index = np.array(reduce(lambda a,b:a+b,index_df['val'].values))
    test_index = np.array(reduce(lambda a,b:a+b,index_df['test'].values))


    train_x,train_y = X_hr[train_index],y_participant[train_index]
    val_x,val_y = X_hr[val_index],y_participant[val_index]
    test_x,test_y = X_hr[test_index],y_participant[test_index]
    return train_x,train_y,val_x,val_y,test_x,test_y,train_index,test_index,val_index 

In [3]:
import pandas as pd
window_size = 8
n_timesteps = window_size*32
n_channels = 3
train_x,train_y,val_x,val_y,test_x,test_y,train_index,test_index,val_index  = get_train_test_val_data(X[:,:,:],
                                                                                                      y_participant,
                                                                                                      y_activity,
                                                                                                      y_time)
train_y = y_activity[train_index]
test_y = y_activity[test_index]
val_y = y_activity[val_index]

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,LeaveOneGroupOut
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.model_selection import train_test_split
# import numpy as np
# %matplotlib notebook
from tensorflow.keras.layers import Conv1D,BatchNormalization,Dropout,Input,MaxPooling1D,Flatten,Dense,Input
from tensorflow.keras.models import Model, Sequential

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

4 Physical GPUs, 1 Logical GPU


In [5]:
def get_model(input_shape=(256,3),act='tanh',loss="mae",opt='adam',n_classes=1):
    model =  Sequential()
    model.add(Conv1D(100,10,input_shape=input_shape,activation='linear',kernel_initializer='normal',padding='same'))
    model.add(Conv1D(100,10,input_shape=input_shape,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(100,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(100,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Conv1D(200,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(50,2,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(4))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Flatten())
    model.add(Dense(n_classes*20,activation=None,kernel_initializer='normal'))
    model.add(tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1),name='embedding'))
    model.add(Dense(n_classes,activation='sigmoid',kernel_initializer='normal'))
    model.add(Dense(n_classes,activation='softmax',kernel_initializer='normal'))
    model.compile(loss=loss,optimizer=opt)
    return model
model =  get_model(n_classes=6)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 256, 100)          3100      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 256, 100)          100100    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 128, 100)          0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 100)          400       
_________________________________________________________________
dropout (Dropout)            (None, 128, 100)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 128, 100)          100100    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 64, 100)           0

In [ ]:
from keras.models import load_model
n_classes = len(np.unique(y_activity))
model = get_model(n_classes=n_classes,loss='sparse_categorical_crossentropy')
filepath = '../model_files/activity_estimator_8_secs_dalia.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=40)
callbacks_list = [es,checkpoint]
history = model.fit(train_x,train_y,validation_data=(val_x,val_y), epochs=100, batch_size=10,verbose=1,
      callbacks=callbacks_list,shuffle=True)

Epoch 1/100
1756/1756 [==============================] - 26s 13ms/step - loss: 1.5940 - val_loss: 1.8919

Epoch 00001: val_loss improved from inf to 1.89192, saving model to ../model_files/activity_estimator_8_secs_dalia.hdf5
Epoch 2/100
1756/1756 [==============================] - 23s 13ms/step - loss: 1.2399 - val_loss: 1.9956

Epoch 00002: val_loss did not improve from 1.89192
Epoch 3/100
1756/1756 [==============================] - 23s 13ms/step - loss: 1.0143 - val_loss: 1.7638

Epoch 00003: val_loss improved from 1.89192 to 1.76380, saving model to ../model_files/activity_estimator_8_secs_dalia.hdf5
Epoch 4/100
1756/1756 [==============================] - 23s 13ms/step - loss: 0.7620 - val_loss: 0.6972

Epoch 00004: val_loss improved from 1.76380 to 0.69717, saving model to ../model_files/activity_estimator_8_secs_dalia.hdf5
Epoch 5/100
1756/1756 [==============================] - 23s 13ms/step - loss: 0.5860 - val_loss: 0.7173

Epoch 00005: val_loss did not improve from 0.69717


In [15]:
model.load_weights(filepath)
from sklearn.metrics import classification_report
print(classification_report(val_y,model.predict(val_x).argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.72      0.99      0.84       451
           1       0.55      0.70      0.62       325
           2       0.79      0.18      0.29       229
           3       1.00      0.39      0.56       345
           4       0.62      0.93      0.74       683
           5       0.86      0.47      0.61       474

    accuracy                           0.68      2507
   macro avg       0.76      0.61      0.61      2507
weighted avg       0.74      0.68      0.65      2507

